## MLDC Mapping (This Project)
Each step below is **working here** in this notebook.

1. **Problem Definition** → Emotion detection from EEG
2. **Data Collection** → Load EEG CSVs from `features_raw.csv`
3. **Data Processing** → Missing values + scaling
4. **EDA** → Shape, samples, statistics
5. **Feature Engineering** → Raw EEG channels (baseline)
6. **Model Selection** → Linear + Logistic Regression
7. **Deployment** → Exported to web UI in this project


### Dataset Used (features_raw.csv)
- 32 EEG channels per row (time samples).
- No real labels included → we generate **synthetic** valence/arousal/dominance.


## 1) Problem Definition
We want to predict emotion dimensions from EEG: **valence**, **arousal**, **dominance**.


## 2) Imports


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score


## 3) Data Loading
Load the EEG data file with channel columns.


In [ ]:
data = pd.read_csv("../features_raw.csv")
# Drop empty column if it exists
if 'Unnamed: 32' in data.columns:
    data = data.drop(columns=['Unnamed: 32'])
data.head()


## 4) EDA (Basic Exploration)


In [ ]:
data.shape


In [ ]:
data.describe().loc[["mean", "std"]].head()


## 5) Preprocessing
- Fill missing values
- Scale features for ML


In [ ]:
data = data.fillna(data.mean())
X = data.values
scaler = StandardScaler()
X = scaler.fit_transform(X)


## 6) Feature Selection / Creation
We use **raw scaled EEG channels** as baseline features.


## 7) Create Synthetic Valence/Arousal/Dominance Labels
Because real emotion labels are missing, we create **pseudo-labels** using EEG patterns.


In [ ]:
# Helper: scale any signal to 1–9 range
def scale_1_9(x):
    return (x - x.min()) / (x.max() - x.min()) * 8 + 1

# Valence: frontal asymmetry (right - left)
valence_raw = (data['F4'] + data['Fp2']) - (data['F3'] + data['Fp1'])

# Arousal: overall absolute activity
arousal_raw = data.abs().mean(axis=1)

# Dominance: central + parietal activity (simple heuristic)
dom_channels = ['C3','C4','P3','P4','Pz']
dominance_raw = data[dom_channels].abs().mean(axis=1)

# Scale to 1–9
y_val = scale_1_9(valence_raw)
y_ar = scale_1_9(arousal_raw)
y_dom = scale_1_9(dominance_raw)

# Binary labels (High vs Low)
y_val_bin = (y_val > y_val.median()).astype(int)
y_ar_bin = (y_ar > y_ar.median()).astype(int)
y_dom_bin = (y_dom > y_dom.median()).astype(int)


## 8) Train/Test Split


In [ ]:
# Continuous splits
X_train, X_test, yv_train, yv_test = train_test_split(X, y_val, test_size=0.2, random_state=42)
_, _, ya_train, ya_test = train_test_split(X, y_ar, test_size=0.2, random_state=42)
_, _, yd_train, yd_test = train_test_split(X, y_dom, test_size=0.2, random_state=42)

# Binary splits
X_train_b, X_test_b, yv_train_b, yv_test_b = train_test_split(X, y_val_bin, test_size=0.2, random_state=42)
_, _, ya_train_b, ya_test_b = train_test_split(X, y_ar_bin, test_size=0.2, random_state=42)
_, _, yd_train_b, yd_test_b = train_test_split(X, y_dom_bin, test_size=0.2, random_state=42)


## 9) Linear Regression (Intensity Scores)


In [ ]:
lin = LinearRegression()

# Valence
lin.fit(X_train, yv_train)
pred_v = lin.predict(X_test)
print('Valence MSE:', mean_squared_error(yv_test, pred_v))

# Arousal
lin.fit(X_train, ya_train)
pred_a = lin.predict(X_test)
print('Arousal MSE:', mean_squared_error(ya_test, pred_a))

# Dominance
lin.fit(X_train, yd_train)
pred_d = lin.predict(X_test)
print('Dominance MSE:', mean_squared_error(yd_test, pred_d))


## 10) Logistic Regression (High vs Low)


In [ ]:
log = LogisticRegression(max_iter=1000)

# Valence High/Low
log.fit(X_train_b, yv_train_b)
pred_vb = log.predict(X_test_b)
print('Valence Accuracy:', accuracy_score(yv_test_b, pred_vb))

# Arousal High/Low
log.fit(X_train_b, ya_train_b)
pred_ab = log.predict(X_test_b)
print('Arousal Accuracy:', accuracy_score(ya_test_b, pred_ab))

# Dominance High/Low
log.fit(X_train_b, yd_train_b)
pred_db = log.predict(X_test_b)
print('Dominance Accuracy:', accuracy_score(yd_test_b, pred_db))
